In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from pathlib import Path
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\

In [2]:
# Import Dataset
# file_path_21 = Path("Resources/batting_stats_2021.csv")
# file_path_20 = Path("Resources/batting_stats_2020.csv")
# file_path_19 = Path("Resources/batting_stats_2019.csv")
# file_path_18 = Path("Resources/batting_stats_2018.csv")
# file_path_17 = Path("Resources/batting_stats_2017.csv")
# file_path_16 = Path("Resources/batting_stats_2016.csv")
# file_path_15 = Path("Resources/batting_stats_2015.csv")
# file_path_14 = Path("Resources/batting_stats_2014.csv")
# file_path_13 = Path("Resources/batting_stats_2013.csv")
# file_path_12 = Path("Resources/batting_stats_2012.csv")
# file_path_11 = Path("Resources/batting_stats_2011.csv")
# file_path_10 = Path("Resources/batting_stats_2010.csv")

# Read into Dataset
# stats_21_df = pd.read_csv(file_path_21, index_col = 0)
# stats_20_df = pd.read_csv(file_path_20, index_col = 0)
# stats_19_df = pd.read_csv(file_path_19, index_col = 0)
# stats_18_df = pd.read_csv(file_path_18, index_col = 0)
# stats_17_df = pd.read_csv(file_path_17, index_col = 0)
# stats_16_df = pd.read_csv(file_path_16, index_col = 0)
# stats_15_df = pd.read_csv(file_path_15, index_col = 0)
# stats_14_df = pd.read_csv(file_path_14, index_col = 0)
# stats_13_df = pd.read_csv(file_path_13, index_col = 0)
# stats_12_df = pd.read_csv(file_path_12, index_col = 0)
# stats_11_df = pd.read_csv(file_path_11, index_col = 0)
# stats_10_df = pd.read_csv(file_path_10, index_col = 0)

# stats_19_df.head(20)

In [3]:
path = 'Resources/batting_stats_*.csv' # use your path
all_files = glob.glob(path)
all_files

data = []

for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0)    
    year = file[-8:-4]
    df['Year'] = year
    data.append(df)

stats_df = pd.concat(data, axis=0, ignore_index=True)
stats_df = stats_df[['Year', 'Player', 'Team', 'Pos', 'Age', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR','RBI', 
             'SB', 'CS', 'BB', 'SO', 'SH', 'SF', 'HBP', 'AVG', 'OBP', 'SLG','OPS']]

stats_df.head()

,Year,Player,Team,Pos,Age,G,AB,R,H,2B,...,CS,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS
0,2010,Ichiro Suzuki,SEA,OF,47,162,680,74,214,30,...,9,45,86,3,1,3,0.315,0.359,0.394,0.753
1,2010,Derek Jeter,NYY,SS,47,157,663,111,179,30,...,5,63,106,1,3,9,0.270,0.340,0.370,0.710
2,2010,Michael Young,TEX,3B,44,157,656,99,186,36,...,2,50,115,0,11,1,0.284,0.330,0.444,0.774
3,2010,Juan Pierre,CWS,OF,43,160,651,96,179,18,...,18,45,47,15,2,21,0.275,0.341,0.316,0.657
4,2010,Rickie Weeks,MIL,DH,38,160,651,112,175,32,...,4,76,184,0,2,25,0.269,0.366,0.464,0.830


In [4]:
# Creating X & y variables
X = stats_df.iloc[:, 4:-2]
y = stats_df["OPS"].values

In [5]:
# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [6]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train.shape

(6069, 18)

In [7]:
#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [12]:
# Model set-up
number_input_features = 87
hidden_nodes_layer1 = 34
hidden_nodes_layer2 = 5

In [13]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid"))

In [14]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

In [15]:
# Training the model
batch_size = 1000
epochs = 50
training_history = model.fit(
    X_train,
    y_train,
    #validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

Epoch 1/50
6069/6069 [==============================] - 0s 29us/sample - loss: 0.8018 - acc: 0.0260 - tp: 569.0000 - tn: 156.0000 - fp: 0.0000e+00 - fn: 5344.0000 - precision: 1.0000 - recall: 0.0962 - auc: 0.5853
Epoch 2/50
6069/6069 [==============================] - 0s 3us/sample - loss: 0.7349 - acc: 0.0260 - tp: 1718.0000 - tn: 156.0000 - fp: 0.0000e+00 - fn: 4195.0000 - precision: 1.0000 - recall: 0.2905 - auc: 0.7109
Epoch 3/50
6069/6069 [==============================] - 0s 3us/sample - loss: 0.7022 - acc: 0.0255 - tp: 3010.0000 - tn: 151.0000 - fp: 5.0000 - fn: 2903.0000 - precision: 0.9983 - recall: 0.5090 - auc: 0.8204
Epoch 4/50
6069/6069 [==============================] - 0s 3us/sample - loss: 0.6872 - acc: 0.0278 - tp: 3872.0000 - tn: 153.0000 - fp: 3.0000 - fn: 2041.0000 - precision: 0.9992 - recall: 0.6548 - auc: 0.9103
Epoch 5/50
6069/6069 [==============================] - 0s 3us/sample - loss: 0.6778 - acc: 0.0285 - tp: 4204.0000 - tn: 156.0000 - fp: 0.0000e+00 - fn:

In [16]:
# Make predictions using the testing data X_test
predicted = model.predict(X_test)

# Evaluate the model
model.evaluate(X_test, y_test)

2023/2023 [==============================] - 0s 46us/sample - loss: 0.5489 - acc: 0.0302 - tp: 1696.0000 - tn: 50.0000 - fp: 0.0000e+00 - fn: 277.0000 - precision: 1.0000 - recall: 0.8596 - auc: 1.0000


[0.5489245710311641,
 0.030153237,
 1696.0,
 50.0,
 0.0,
 277.0,
 1.0,
 0.85960466,
 1.0]

In [17]:
real_OPS = stats_df["OPS"]

In [18]:
moneyball = pd.DataFrame({
    "Real": real_OPS.ravel(),
    "Predicted": predicted.ravel()
    }, index = stats_df.index[-len(real_OPS): ])
moneyball.head()

ValueError: Shape of passed values is (2023, 2), indices imply (8092, 2)